In [ ]:
from agents import manager_agent


In [ ]:
manager_agent.visualize()

In [ ]:
from PIL import Image

# Path to image file
picture_path = './bird_image.png'

# List to hold images
images = []

# Open image from path
image = Image.open(picture_path)

# Ensure the image is loaded properly and append it to the list of images
if image:
    images.append(image.convert('RGB'))

manager_agent.run("How many bird species are in this image?", images=images)

╭─────────────────────────────────────────────── New run - Manager ───────────────────────────────────────────────╮
│                                                                                                                 │
│ How many bird species are in this picture?                                                                      │
│                                                                                                                 │
╰─ OpenAIServerModel - cogito:14b ────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
1. Since there's no image provided with the task question, and we only have access to video and web tools, I need 
to first establish if this is a YouTube video link or something else.
2. Ask team member web_agent to search for an image link or confirm if the context refers to a specific YouTube 
video related to bird species documentation
3. If a YouTube URL is discovered through the web search:
   - Use youtube_frames_to_images tool with appropriate sample interval (e.g., 5 seconds) to generate image frames
4. Once images are obtained, use use_vision_model tool to ask how many distinct bird species can be identified in 
these frames
5. Process vision model's answer carefully and potentially validate or supplement if needed
6. Use final_answer tool to submit the total count of bird species


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 1: Duration 16.19 seconds| Input tokens: 560 | Output tokens: 170]

KeyboardInterrupt: 

In [ ]:
from tools.tools import youtube_frames_to_images

In [ ]:
results = youtube_frames_to_images('https://youtube.com/watch?v=L1vXCYZAYYM')

In [ ]:
import socket
print(socket.gethostbyname("www.youtube.com"))


In [ ]:
results

In [ ]:
manager_agent.run("How many r's are in the word Strawberry?")


In [ ]:
from tools.tools import use_vision_model

use_vision_model("How many bird species are in this picture?", images=images)